In [ ]:
#Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.dates as mdates
import seaborn as sns
#import json
import re
import os

In [ ]:
#Import Alaska data
#Putting cols in lowercase and replace spaces
def df_col_clnr(df):
    for i in df.columns:
        txt = re.sub(" ","_",i)
        txt = txt.lower()
        if txt == "booking_class_of_service":
            txt = "cos"
        df = df.rename(columns={i : txt})
        print(txt)
    return df
df = pd.read_excel("Assessment_RM.xlsx")
df = df_col_clnr(df)
df["cos_cnt"]=[1 for i in range(len(df))]

In [ ]:
def cos_graph(var,x,y,filt,title):
#INPUTS:
    #var:dataframe
    #x:dependent
    #y:independent
    #filt:booking cos
#OUTPUTS:
    #plot for trend line
    temp_df = var
    if filt != None:
        temp_df = var[var['cos'] == filt]
    temp_df = temp_df.sort_values(x, ascending=False)
    sns.set_style('dark')
    plt.figure(figsize = (15,10))
    ax = sns.barplot(x = x, y = y, 
                     data = temp_df, palette = 'cool')
    #for container in ax.containers:
    #    ax.bar_label(container)
    
    plt.title('correlation of: {} and {}'.format(x,y))
    plt.xticks(rotation = 45)
    #plt.savefig("{}.jpg".format(title))
    plt.show()

In [ ]:
##1)SHOWS lin trend between tickets sold and revenue earned (linear correlation)
total_tx_df = pd.DataFrame({"flight_date":df["flight_date"], "total_tc_cnt":df["cos_cnt"]})
total_tx_df = total_tx_df.groupby(total_tx_df["flight_date"]).sum()
total_tx_df = total_tx_df.reset_index(drop=False)

total_rev_df = pd.DataFrame({"flight_date":df["flight_date"],
                       "total_rev":df.fare_amount})
total_rev_df = total_rev_df.groupby(total_rev_df["flight_date"]).sum()
total_rev_df = total_rev_df.reset_index(drop=False)

merged_total_rev_df = pd.merge(total_tx_df, total_rev_df, on='flight_date')

In [ ]:
##2)SHOWS lin trend between total_rev and by cos total rev
total_rev_df = pd.DataFrame({"flight_date":df["flight_date"],
                       "total_rev":df.fare_amount})
total_rev_df = total_rev_df.groupby(total_rev_df["flight_date"]).sum()
total_rev_df = total_rev_df.reset_index(drop=False)

cos_rev_sum = pd.DataFrame({"flight_date":df.flight_date,
                       "cos":df.cos,
                       "cos_rev_sum":df.fare_amount
                      })

cos_rev_sum = cos_rev_sum.groupby(["flight_date","cos"]).sum()
cos_rev_sum = cos_rev_sum.reset_index(drop=False)

merged_tr_cr_df = pd.merge(total_rev_df, cos_rev_sum, on='flight_date')

In [ ]:
##3)SHOWS lin trend between cos_cnt and sum by cos rev
cos_cnt = pd.DataFrame({"flight_date":df.flight_date,
                       "cos":df.cos,
                       "cos_cnt":df.cos_cnt
                      })

cos_cnt = cos_cnt.groupby(["flight_date","cos"]).sum()

cos_cnt = cos_cnt.reset_index(drop=False)

merged_cos_sub_df = pd.merge(cos_cnt, cos_rev_sum, on=['flight_date','cos'])

In [ ]:
##1)SHOWS lin trend between tickets sold and revenue earned (linear correlation)
#dependent total_tc_cnt
#independent total_rev
cos_graph(merged_total_rev_df,"total_tc_cnt","total_rev",None,"total_tc_rev_trend")

In [ ]:
##2)SHOWS lin trend between total_rev and by cos total rev
#dependent total_rev
#independent cos_rev_sum
cos_graph(merged_tr_cr_df,"total_rev","cos_rev_sum","N","N_cos_v_total_rev_trend")

In [ ]:
##3)SHOWS lin trend between cos_cnt and sum by cos rev
#dependent cos_rev_sum
#independent cos_cnt
cos_graph(merged_cos_sub_df,"cos_rev_sum","cos_cnt","N","N_cos_cnt_trend")